In [7]:
import praw
import pandas as pd
import time
from datetime import datetime, timedelta

# --- Logging ---
def log(msg):
    print(msg)

def timestamp():
    return datetime.now().strftime('%H:%M:%S')

# --- Reddit API setup ---
reddit = praw.Reddit(
    client_id="0GgjEdXVoxIm0LTkBxIjNA",
    client_secret="NsY4t_AN6o5E4SSki7YkhVee07DCzw",
    user_agent="game-review-bot"
)

# --- Game and keywords ---
game_name = "Assassin's Creed Shadows"
subreddit = "games"  # only one subreddit

keywords = [
    "assassin's creed shadows"]

# --- Date range: daily ---
start_date = datetime(2025, 2, 28)
end_date = datetime(2025, 3, 31)
step = timedelta(days=1)

# --- Storage ---
posts_data = []
comments_data = []
deduped_post_ids = set()

# --- Daily scraping loop ---
current_start = start_date
while current_start < end_date:
    current_end = current_start + step
    start_ts = int(current_start.timestamp())
    end_ts = int(current_end.timestamp())

    log(f"[{timestamp()}] 🔍 Scraping for {game_name} in r/{subreddit} from {current_start.date()} to {current_end.date()}")

    try:
        for kw in keywords:
            for post in reddit.subreddit(subreddit).search(kw, sort="new", time_filter="all", limit=100):
                time.sleep(1)  # reduced lag, but still respectful

                if not (start_ts <= post.created_utc < end_ts):
                    continue
                if post.id in deduped_post_ids:
                    continue

                full_text = f"{post.title} {post.selftext}".lower()
                title_body_keywords = [k for k in keywords if k in full_text]
                is_strong_match = "assassin's creed shadows" in full_text

                matched_in_comments = False
                comment_matches = []

                try:
                    post.comments.replace_more(limit=None)
                    all_comments = post.comments.list()
                    log(f"    🗨️ {len(all_comments)} comments in post {post.id}")

                    for i, comment in enumerate(all_comments[:300]):  # optional cap
                        try:
                            comment_text = comment.body.lower()
                            if any(k in comment_text for k in keywords):
                                matched_in_comments = True
                                comment_matches.append({
                                    "game": game_name,
                                    "keyword": kw,
                                    "subreddit": subreddit,
                                    "post_id": post.id,
                                    "comment_id": comment.id,
                                    "comment_body": comment.body,
                                    "author": str(comment.author) if comment.author else "deleted",
                                    "score": comment.score,
                                    "created_utc": comment.created_utc
                                })
                        except Exception as e:
                            log(f"⚠️ Skipped bad comment in post {post.id}: {e}")
                            continue

                except Exception as e:
                    log(f"⚠️ Error fetching comments for post {post.id}: {e}")

                if is_strong_match or len(title_body_keywords) >= 2 or matched_in_comments:
                    deduped_post_ids.add(post.id)
                    posts_data.append({
                        "game": game_name,
                        "keyword": kw,
                        "subreddit": subreddit,
                        "post_id": post.id,
                        "title": post.title,
                        "selftext": post.selftext,
                        "score": post.score,
                        "num_comments": post.num_comments,
                        "created_utc": post.created_utc,
                        "author": str(post.author),
                        "matched_keywords_title_body": title_body_keywords
                    })
                    comments_data.extend(comment_matches)

    except Exception as e:
        log(f"⛔ Error scraping r/{subreddit} on {current_start.date()} → {e}")

    current_start = current_end  # move to next day


[12:32:00] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-02-28 to 2025-03-01
    🗨️ 1 comments in post 1j0csmz
[12:33:42] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-01 to 2025-03-02
[12:35:23] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-02 to 2025-03-03
[12:37:05] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-03 to 2025-03-04
[12:38:46] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-04 to 2025-03-05
    🗨️ 68 comments in post 1j3op9z
    🗨️ 23 comments in post 1j3h9uk
[12:40:28] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-05 to 2025-03-06
[12:42:10] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-06 to 2025-03-07
    🗨️ 19 comments in post 1j4z95r
[12:43:51] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-07 to 2025-03-08
[12:45:32] 🔍 Scraping for Assassin's Creed Shadows in r/games from 2025-03-08 to 2025-03-09
[12:47:13] 🔍 Scraping for Assassi

In [9]:
posts_df = pd.DataFrame(posts_data).drop_duplicates(subset="post_id")
comments_df = pd.DataFrame(comments_data)

posts_df.to_csv("acs_games_posts331.csv", index=False)
comments_df.to_csv("acs_games_comments331.csv", index=False)

log(f"✅ Done! Saved {len(posts_df)} posts and {len(comments_df)} comments.")


✅ Done! Saved 27 posts and 17 comments.
